In [4]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torchvision.transforms as transforms

from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, TensorDataset

from torch import nn

ModuleNotFoundError: No module named 'torchvision'

In [ ]:
torch.__version__

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

## 데이터셋 불러오기

In [ ]:
df = pd.read_csv("all_data.csv")

# target 열에 데이터가 있으면 train, 없으면 final 나누기
train = df[df.target.notnull()]
final = df[df.target.isnull()]

In [ ]:
#test and split 임포트
from sklearn.model_selection import train_test_split

#데이터셋 분리
train, test = train_test_split(train, test_size=0.2)

# Prepare data for DataLoader
X_train = torch.tensor(train.drop('target', axis=1).values, dtype=torch.float32)
y_train = torch.tensor(train['target'].values, dtype=torch.long)
X_test = torch.tensor(test.drop('target', axis=1).values, dtype=torch.float32)
y_test = torch.tensor(test['target'].values, dtype=torch.long)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

In [ ]:
#데이터로더 만들기
train_loader = DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=128, shuffle=False)

## Fully Connected Model

In [ ]:
# 28x28x1 => 10
#sequential 순차적으로 블럭을 쌓아서 만든다. 플래튼부터 소프트맥스까지 하나씩 거쳐가는 형식이다.
#보통은 시퀀셜이 아닌 클래스로 모델을 만든다.
mnist_fc_model = nn.Sequential(
    nn.Flatten(), #28x28x1을 784로 펼치는 역할 내가 넣으려는 데이터는 2차원이 아니므로 필요 없을듯
    nn.Linear(in_features=48, out_features=256), #in_features, out_features를 맞추주는 것이 중요
    nn.ReLU(), #요즘은 렐루, 리켈루를 많이 쓴다. 무난하게는 ReLU함수 사용
    nn.Linear(in_features=256, out_features=2), #나의 경우는 아웃피처가 2가 되어야함(정상 불량)
    nn.Softmax() #꼭 해줘야함. 확률값으로 바꿔주는 것
)
#다항함수 하나가 Linear고 2개를 쌓은 모습이다.

# Model definition
model = nn.Sequential(
    nn.Linear(in_features=X_train.shape[1], out_features=256), 
    nn.ReLU(),
    nn.Linear(in_features=256, out_features=2)  # Assuming binary classification
)

In [ ]:
# 만든 모델을 device(cuda)로 전달하자
#device는 위에 cuda, cpu 부분 참고
#연산을 GPU의 VRAM으로 모델을 보내줌 데이터는 아직 안보냄 후에 배치로 데이터를 보낼 것이다.
mnist_fc_model.to(device)

### 실제 학습이 일어나는 train() 함수

In [2]:
from torch import optim
import time

ImportError: cannot import name 'inf' from 'torch' (unknown location)

In [ ]:
def train(model, train_loader):
    epochs = 10
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    criterion = nn.CrossEntropyLoss()
    
    start_time = time.time()
    for epoch in range(epochs):
        model.train()
        print(f'epochs:{epoch + 1} / {epochs}')
        
        for x_t, y_t in train_loader:
            x_t = x_t.to(device)
            y_t = y_t.to(device)
            
            pred = model(x_t)
            loss = criterion(pred, y_t)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        correct = 0
        # Evaluate model on test_loader
        for xx, yy in test_loader:  
            xx = xx.to(device)
            yy = yy.to(device)
            pred = model(xx)
            _, predicted = torch.max(pred, 1)
            correct += predicted.eq(yy.data).sum()

        print(f'test_accuracy: {(100. * correct / len(test_loader.dataset)).item()}')

    torch.cuda.synchronize()
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f'Total training time: {elapsed_time // 60} minutes {elapsed_time % 60} seconds.')

In [ ]:
train(mnist_fc_model, train_loader)